In [1]:
import numpy as np

from ge.classify import read_node_label, Classifier
from ge import DeepWalk
import networkx as nx

import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc='pandas bar')

pd.set_option('max_columns', None)
pd.set_option('max_rows', 300)
pd.set_option('float_format', lambda x: '%.3f' % x)

tgt_market = 't1'
src_market = 's1'
tgt_data_dir = f'./DATA/{tgt_market}/'

2021-12-09 09:06:42.971613: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# 读取训练数据
train = pd.read_table(tgt_data_dir+'train.tsv')
train_5core = pd.read_table(tgt_data_dir+'train_5core.tsv')

train_cross = train_5core.merge(train, on=['userId', 'itemId'], how='left')
train_5core = train_cross[train_cross['rating_y'].isnull()][['userId', 'itemId']]

train = pd.concat([train, train_5core, train_5core, train_5core, train_5core, train_5core], ignore_index=True)[['userId', 'itemId']]

In [3]:
G = nx.from_pandas_edgelist(train, source='userId', target='itemId')

embed_size = 32
model = DeepWalk(G, walk_length=5, num_walks=80, workers=8)
model.train(embed_size=embed_size, window_size=3, iter=3)
embeddings = model.get_embeddings()

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
2021-12-09 09:06:49.509981: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-12-09 09:06:50.953420: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-12-09 09:06:52.802128: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-12-09 09:06:54.223123: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-12-09 09:06:55.154779: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-12-09 09:06:56.260762: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-12-09 09:06:57.512813: I tensorflow/stream_e

Learning embedding vectors...
Learning embedding vectors done!


In [4]:
embed_df = pd.DataFrame(columns=['id'])
embed_df['id'] = list(embeddings.keys())
# embed_df['emb'] = list(embeddings.values())

emb_matrix = np.array(list(embeddings.values()))
for i in range(embed_size):
    embed_df['deepwalk_emb_{}'.format(i)] = emb_matrix[:, i]

In [5]:
embed_df.to_csv(f'./deepwalk_{tgt_market}.csv', index=None)